In [ ]:
import ray
ray.init()

 

In [ ]:
items = [{"name": str(i), "data": i} for i in range(10000)]
ds = ray.data.from_items(items)   
ds.show(5) 

In [ ]:
squares = ds.map(lambda x: x["data"] ** 2)  

evens = squares.filter(lambda x: x % 2 == 0)
evens.count()

cubes = evens.flat_map(lambda x: [x, x**3])  
sample = cubes.take(10)  
print(sample)

In [ ]:
# tag::ray_data_pipeline[]
pipe = ds.pipeline()  # <1>
result = pipe\
    .map(lambda x: x["data"] ** 2)\
    .filter(lambda x: x % 2 == 0)\
    .flat_map(lambda x: [x, x**3])  # <2>
result.show(10)
# end::ray_data_pipeline[]

In [ ]:
# tag::ray_tune[]
from ray import tune
import math
import time


def training_function(config):  # <1>
    x, y = config["x"], config["y"]
    time.sleep(10)
    score = objective(x, y)
    tune.report(score=score)  # <2>


def objective(x, y):
    return math.sqrt((x**2 + y**2)/2)  # <3>


result = tune.run(  # <4>
    training_function,
    config={
        "x": tune.grid_search([-1, -.5, 0, .5, 1]),  # <5>
        "y": tune.grid_search([-1, -.5, 0, .5, 1])
    })

print(result.get_best_config(metric="score", mode="min"))
# end::ray_tune[]